In [17]:
import pandas as pd
import numpy as np
import torch
torch.set_default_dtype(torch.float64)


batch_size = 64
epochs = 5

#Ben Directories
sensor_geom_path='/opt/app/data/erdos-data/sensor_geometry.csv'
batch_path='/opt/app/data/erdos-data/train/batch_104.parquet'

In [18]:
# Without considerations for the auxiliary label and no explicit features consisting of sensor geometry
from torch.utils.data import Dataset, DataLoader

# Class for a dataset generated from a dataframe and data from the sensor geometry file
class NeutrinoDataset(Dataset):
    def __init__(self, filename):
        self.sensor_geom = pd.read_csv(sensor_geom_path)
        self.vals_df = pd.read_parquet('batch_104_directions.parquet')
        self.dataframe = pd.read_parquet(filename)
        sensor_loc = np.array(sg.iloc[:])[:, 1:]
        self.num_features = 5160*3
        self.num_events = self.dataframe.index.nunique()
        self.unique_indices = np.unique(self.dataframe.index)
        
    def __len__(self):
        return self.num_events
    
    # Replaces sensor ID with sensor coordinates 
    def __getitem__(self, i):
        df = self.dataframe
        sg = self.sensor_geom
        meta_vals = np.array(
            self.vals_df.loc[self.vals_df['event_id'] == df.index[i]])[0].astype(float)
        
        pulse_array = np.array(df.loc[df.index[i]])
        pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 3])), 1)

        for pulse in pulse_array:
            if(pulse_array_sensors[pulse[0]][1] == 0):
                pulse_array_sensors[pulse[0]][1] = pulse[1] - meta_vals[2] # first time
            else:
                # possible last time, will be the last time for the actual last one
                pulse_array_sensors[pulse[0]][2] = pulse[1] - meta_vals[2]
            # Add charge
            pulse_array_sensors[pulse[0]][3] += pulse[2]
        
        flattened_pulse = (pulse_array_sensors[:, 1:]).flatten()
        # print(flattened_pulse.shape)
                
        return (torch.from_numpy(flattened_pulse), 
                                 torch.from_numpy(meta_vals[-2:]))
    
    # Finds the first event with multiple pulses at the same sensors
    # here we ask for at least num_min_total_repeats repetitions
    def get_multi_pulse_event(self, num_min_total_repeats):
        for i in range(self.num_events):
            pulses = np.array(df.loc[unique_indices[i]])
            if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
                return self.unique_indices[i]
            
    # Finds all events in a range with multiple pulses at the same sensors
    # here we ask for at least num_min_total_repeats repetitions
    def get_multi_pulse_events(self, num_min_total_repeats, start_index, end_index):
        list_multi_pulse = []
        for i in range(start_index, min(self.num_events, end_index)):
            pulses = np.array(df.loc[unique_indices[i]])
            if(pulses[:,0].shape[0] - np.unique(pulses[:,0]).shape[0] >= num_min_total_repeats):
                list_multi_pulse.append(self.unique_indices[i])
        return list_multi_pulse


In [19]:
# Checking torch device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [20]:
# Set up Dataset and DataLoader, build NN

import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.utils.data import Dataset, DataLoader

sg = pd.read_csv(sensor_geom_path)

dataset = NeutrinoDataset(batch_path)


class NNPredictor(torch.nn.Module):
    def __init__(self, use_activation = True):
        super().__init__()
        # torch.manual_seed(1234)
        self.layers = nn.ModuleList()
        self.layer_norms = nn.ModuleList()
        self.use_activation = use_activation
        
        self.layers.append(nn.Linear(dataset.num_features, 100))
        self.layers.append(nn.Linear(100, 50))
        self.layers.append(nn.Linear(50, 10))
        self.classifier = (nn.Linear(10,2))

    def forward(self, x):
        new_x = x
        if(self.use_activation):
            for layer in self.layers:
                # print(layer, new_x.shape)
                new_x = layer(new_x)
                new_x = nn.ReLU()(new_x)
        else:
            for layer in self.layers:
                new_x = layer(new_x)
        
        # Apply a final (linear) classifier.

        return self.classifier(new_x)


2.1.0+cu121


In [21]:

model = NNPredictor()

In [22]:
batch_104_vals_df = pd.read_parquet('batch_104_directions.parquet')

In [35]:
def angular_dist_score(predictions, true):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    az_true=true[0]
    zen_true=true[1]
    az_pred=predictions[0]
    zen_pred=predictions[1]
    
    if not (torch.all(torch.isfinite(az_true)) and
            torch.all(torch.isfinite(zen_true)) and
            torch.all(torch.isfinite(az_pred)) and
            torch.all(torch.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = torch.sin(az_true)
    ca1 = torch.cos(az_true)
    sz1 = torch.sin(zen_true)
    cz1 = torch.cos(zen_true)
    
    sa2 = torch.sin(az_pred)
    ca2 = torch.cos(az_pred)
    sz2 = torch.sin(zen_pred)
    cz2 = torch.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  torch.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return torch.mean(torch.abs(torch.arccos(scalar_prod)))

In [36]:
class custom_MAE(nn.Module):
    def __init__(self):
        super(custom_MAE, self).__init__();

    def forward(self, predictions, target):
        loss_value = angular_dist_score(predictions, target)
        return loss_value

In [37]:

learning_rate = 1e-9

loss_fn = custom_MAE()


dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # print(X)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 1 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            


In [38]:
train_loop(dataloader, model, loss_fn, optimizer)

loss: 1.379610  [   32/200000]
loss: 0.946007  [   64/200000]
loss: 1.204563  [   96/200000]
loss: 1.899619  [  128/200000]
loss: 0.929960  [  160/200000]
loss: 1.147071  [  192/200000]
loss: 1.094320  [  224/200000]
loss: 1.214869  [  256/200000]
loss: 0.516315  [  288/200000]
loss: 0.596543  [  320/200000]
loss: 1.074611  [  352/200000]
loss: 1.633885  [  384/200000]
loss: 1.306444  [  416/200000]
loss: 1.687012  [  448/200000]
loss: 0.748770  [  480/200000]
loss: 0.917006  [  512/200000]
loss: 2.011028  [  544/200000]
loss: 1.468147  [  576/200000]
loss: 1.345124  [  608/200000]
loss: 1.712065  [  640/200000]
loss: 1.556729  [  672/200000]
loss: 1.574315  [  704/200000]
loss: 1.436975  [  736/200000]
loss: 0.772764  [  768/200000]
loss: 2.050138  [  800/200000]
loss: 1.372484  [  832/200000]
loss: 1.205266  [  864/200000]
loss: 0.612417  [  896/200000]
loss: 1.559798  [  928/200000]
loss: 0.974482  [  960/200000]
loss: 0.610495  [  992/200000]
loss: 1.331641  [ 1024/200000]
loss: 0.

KeyboardInterrupt: 

In [ ]:
# i=0
# meta_vals = np.array(batch_104_vals_df.loc[batch_104_vals_df['event_id'] 
#                                            == df.index[0]])[0].astype(float)
        
# pulse_array = np.array(df.loc[df.index[i]])
# pulse_array_sensors = np.concatenate((np.expand_dims(np.arange(5160), axis=1), np.zeros([5160, 3])), 1)

# for pulse in pulse_array:
#     if(pulse_array_sensors[pulse[0]][1] == 0):
#         pulse_array_sensors[pulse[0]][1] = pulse[1] - meta_vals[2] # first time
#     else:
#         # possible last time, will be the last time for the actual last one
#         pulse_array_sensors[pulse[0]][2] = pulse[1] - meta_vals[2]
#     # Add charge
#     pulse_array_sensors[pulse[0]][3] += pulse[2]

# print(torch.from_numpy(np.concatenate(
#     (np.ndarray.flatten(pulse_array_sensors[:, 1:]), meta_vals[-2:]))))

In [54]:
loss_total = 0
num = 0
with torch.no_grad():
    for batch, (X, y) in enumerate(dataloader):
        # print(X)
        # Compute prediction and loss
        pred = np.array(model(X))
        y = np.array(y)
        loss_total += angular_dist_score(y[0], y[1], pred[0], pred[1])
        num +=1
        print(loss_total/num)



0.7386982244869325
1.0650035905634065
1.3417156555555945
1.4730707044898226
1.3924171977633228
1.4721141445653219
1.4830690410494232
1.470363600995138
1.5334018653103512
1.5735355753053124
1.5645000481634719
1.6330845664236076
1.65312866652555
1.598865237073891
1.5528595768506936
1.5781901293591631
1.5610519613869396
1.5757805201572639
1.602080908501265
1.5766664047995897
1.5750081957786746
1.5545708645505676
1.5747693584513602
1.5934875309918348
1.6113250947012483
1.6240856069012108
1.6056152847478444
1.56305852801106
1.5718101375047544
1.615236531500848
1.6282648729784444
1.5986009119889812
1.569299928977477
1.5552178964019945
1.5579168600255013
1.5583911958286982
1.542998378330522
1.5415232607956573
1.5630536148069154
1.573990463576131
1.5629801134730839
1.5628833075891115
1.5383684304846361
1.5332331651338713
1.528977149286577
1.5351189543287431
1.523000570331043
1.5376928455547125
1.5569109908416001
1.538880775476145
1.526095839755001
1.5443503164309829
1.5364453464749328
1.544565

KeyboardInterrupt: 